In [ ]:
!pip install transformers elasticsearch

import numpy as np
from transformers import AutoTokenizer, AutoModel
from elasticsearch import Elasticsearch
import torch

# 인증정보를 활용하여 엘라스틱서치 커넥션 정보를 정의합니다. 
es = Elasticsearch(
    ['https://hostname:port'],
    http_auth=('username', 'password'),
    verify_certs=False
)

# Ddense vector field를 위한 맵핑을 정의합니다. 
mapping = {
    'properties': {
        'embedding': {
            'type': 'dense_vector',
            'dims': 768   # Ddense vector field의 차원을 정의합니다. 
        }
    }
}

# 정의한 맵핑으로 인덱스를 생성합니다. 
es.indices.create(index='chapter-2', body={'mappings': mapping})

# 색인 할 데이터셋을 구성합니다.
docs = [
    {
        'title': 'Document 1',
        'text': 'This is the first document.'
    },
    {
        'title': 'Document 2',
        'text': 'This is the second document.'
    },
    {
        'title': 'Document 3',
        'text': 'This is the third document.'
    }
]

# BERT 토크나이저와 모델을 로드합니다. 
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# BERT를 활용하여 각 문서의 임베딩을 생성합니다. 
for doc in docs:
    text = doc['text']
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        output = model(**inputs).last_hidden_state.mean(dim=1).squeeze(0).numpy()
    doc['embedding'] = output.tolist()

# 엘라스틱서치에 문서를 색인합니다. 
for doc in docs:
    es.index(index='chapter-2', body=doc)
